Nama: Irene Maritsa

NIM: H071191026

Pelajari Chapter 5 dari d2l.ai : Deep Learning Computation

In [1]:
import tensorflow as tf
from tensorflow import keras

# 1. Layer dan Blocks

Layer merupakan arsitektur atau struktur yang berisi kumpulan node yang beroperasi secara dan di waktu yang bersamaan, yang mengambil informasi dari layer sebelumnya dan kemudian meneruskan informasi ke layer berikutnya.

Block atau neural network blocks merupakan konsep di mana satu atau lebih layer digambarkan / diabstraksi menjadi satu komponen (block). Salah satu manfaat utama dari abstraksi blocks adalah blocks ini bisa lagi dikombinasikan menjadi struktur yang lebih besar (biasanya secara rekursif).

Dengan menggunakan konsep blocks ini, kita tetap dapat menulis kode yang ringkas dan masih mengimplementasikan neural net yang kompleks. Untuk mengimplementasikan blocks, kita membuat fungsi atau jika ingin fleksibel, kelas turunan dari tensorflow.keras.Model yang memiliki fungsi forward propagation (fungsi call()) . 

Contoh implementasi kode


In [2]:
# Data: digenerate secara random
X = tf.random.uniform((2, 20))

X

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.02576447, 0.20197976, 0.70219517, 0.01233876, 0.59934187,
        0.77002716, 0.38943923, 0.47400331, 0.27422714, 0.2088083 ,
        0.02645361, 0.4790585 , 0.5381237 , 0.4185362 , 0.78749764,
        0.8413707 , 0.80760527, 0.16672003, 0.49980927, 0.7184535 ],
       [0.12169123, 0.19905615, 0.9611039 , 0.5702108 , 0.31353152,
        0.5062885 , 0.9863285 , 0.9086404 , 0.69162047, 0.07209623,
        0.5726782 , 0.7550218 , 0.08292532, 0.9966459 , 0.9974885 ,
        0.82730675, 0.24289107, 0.7210177 , 0.67363083, 0.4886924 ]],
      dtype=float32)>

Misalnya model yang akan dijadikan blok merupakan model seperti ini

In [3]:
block1 = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])

Bentuk bloknya

In [4]:
class Block1(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_list= []
        self.layer_list.append(keras.layers.Dense(256, activation=tf.nn.relu))
        self.layer_list.append(keras.layers.Dense(10))
        self.layer_list.reverse()
    
    def call(self, X):
        result_temp= X

        # Melakukan forward propagation sampai layer terakhir dalam list 
        while bool(self.layer_list):
            result_temp= self.layer_list.pop()(result_temp)

        return result_temp
        # return 1

In [5]:
net= Block1()
net(X) # Sama saja dengan net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.19286352, -0.18749547,  0.03211523,  0.07317674,  0.17570707,
        -0.26441398, -0.06995555,  0.10220586, -0.01584365,  0.10774392],
       [ 0.2363228 , -0.04864379,  0.10196164,  0.01020926,  0.20883676,
        -0.1298388 , -0.14530724,  0.24394602, -0.09210241, -0.05063185]],
      dtype=float32)>

Selanjutnya kita akan membuat block ke dua yang tidak mengandung layer satu per satu tapi mengandung model

In [6]:
class Block2(keras.Model):
    def __init__(self):
        super().__init__()
        self.net= keras.Sequential()
        self.net.add(keras.layers.Dense(256, activation=tf.nn.relu))
        self.net.add(keras.layers.Dense(10))
    
    def call(self, X):
        return self.net(X)

Kemudian satukan

In [7]:
net= keras.Sequential()

# Menambahkan layer pertama yang mana merupakan block 2
net.add(Block2())

# Menambahkan block 1
net.add(Block1())

# # Menambahkan satu lagi layer dense biasa
net.add(keras.layers.Dense(10, activation= 'sigmoid'))

# Melakukan forward propagation
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.50911033, 0.5716525 , 0.51420814, 0.5687796 , 0.56056327,
        0.4985788 , 0.46110842, 0.4469462 , 0.5803917 , 0.50603056],
       [0.49454978, 0.51718116, 0.45687127, 0.50363666, 0.49906126,
        0.48918095, 0.42328098, 0.49218008, 0.59554696, 0.5301467 ]],
      dtype=float32)>

# 2. Parameter Management

Parameter management atau manajemen parameter termasuk (Parameter dalam kasus neural network contohnya weight dan bias, fungsi loss atau learning rate merupakan hyperparameter):

Mengakses parameter untuk debugging, diagnostik, dan visualisasi.
Inisialisasi parameter.
Berbagi parameter dengan komponen model lain.



## Mengakses Parameter

Model:

In [8]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.24649215],
       [0.25672632]], dtype=float32)>

Mengambil weight dan bias dari setiap layer net

In [9]:
net.weights
# Atau net.get_weights()

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.25658697,  0.665904  , -0.43578154,  0.709652  ],
        [ 0.13771528,  0.41275877, -0.78659695, -0.4558903 ],
        [ 0.25031918,  0.08139366,  0.7626386 ,  0.17768484],
        [-0.198075  ,  0.42781276,  0.6238579 , -0.4812101 ]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_10/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[-0.7786403 ],
        [ 0.46571636],
        [ 0.39268887],
        [-0.5590628 ]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

Untuk mendapatkan parameter pada layer yang ingin diakses, dilakukan dengan memanggil atribut .layers[index dari layer] dari model sequential. Misal layer yang ingin diakses parameternya yaitu layer indeks ke-1

net.layers[1]

Selanjutnya untuk mengakses parameter weight dan bias

In [10]:
net.layers[1].weights

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.25658697,  0.665904  , -0.43578154,  0.709652  ],
        [ 0.13771528,  0.41275877, -0.78659695, -0.4558903 ],
        [ 0.25031918,  0.08139366,  0.7626386 ,  0.17768484],
        [-0.198075  ,  0.42781276,  0.6238579 , -0.4812101 ]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

Outputnya berupa array, indeks ke-0 merupakan weights, indeks ke-1 merupakan bias

Untuk layer yang berupa block (memiliki layer lagi di dalamnya), kita tetap bisa mengakses nya seperti ini:

net.layers[0].layers[1].layers[1].weights[1]

### Inisialisasi Parameter

Secara default (untuk tensorflow), matriks weight diinisialisasikan berdasarkan range yang dikomputasi dari dimensi input dan output, sementara parameter bias mempunyai nilai default 0.

Misal menginisialisasi weight sebagai variabel random Gaussian dengan standar deviasi 0.01, dan menginisialisasi parameter bias dengan nilai 0.

In [18]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[1].weights[1]

(<tf.Variable 'dense_20/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00250478, -0.00729429,  0.01993616,  0.01713971],
        [ 0.00932723,  0.00861306,  0.00037473,  0.01281009],
        [ 0.00137298, -0.01202288,  0.01145193, -0.02086713],
        [-0.00207057, -0.00026267,  0.00353656,  0.00783457]],
       dtype=float32)>,
 <tf.Variable 'dense_20/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

Kita juga bisa menginisialisasikan weight yang berbeda pada setiap layer, misal weight layer dense kedua diisi dengan angka konstan 0.5.

In [12]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=keras.initializers.Constant(0.5), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[2].weights[0]

(<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00662467,  0.01239068,  0.01816804,  0.00183871],
        [ 0.00718059,  0.00218495,  0.01927559, -0.0112583 ],
        [ 0.00452408,  0.01702096, -0.01680993, -0.00029011],
        [-0.00881657, -0.00371246, -0.00372501,  0.01336032]],
       dtype=float32)>,
 <tf.Variable 'dense_14/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5]], dtype=float32)>)

Kita juga bisa membuat subclass kita sendiri untuk inisialisasi weight jika cara di atas kurang flexibel, caranya yaitu kita membuat class subclass dari keras.initializers.Initializer , lalu mengimplementasikan method call(shape, dtype) yang me-return tensor (berisi weight inisial) yang isinya sesuai kemauan kita

In [13]:
# Di sini kita membuat class untuk inisialisasi weight
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

In [14]:
MyInit().__call__(shape=(3, 3), dtype=tf.float32)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 8.941259 ,  0.       ,  0.       ],
       [-0.       , -5.9314084, -7.18122  ],
       [-0.       ,  7.7063065,  0.       ]], dtype=float32)>

In [15]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    # Memasukkan kelas initializer pada parameter kernel_initializer
    keras.layers.Dense(4, activation=tf.nn.relu, kernel_initializer=MyInit()),
    keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_16/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-7.334926 , -0.       ,  5.315795 ,  5.652321 ],
       [ 7.428871 , -5.103283 , -0.       ,  0.       ],
       [ 6.934576 , -7.4474382,  5.0421476, -0.       ],
       [ 0.       , -0.       ,  0.       , -0.       ]], dtype=float32)>


# 3. Deferred Initialization
Inisialisasi weight pada model ditangguhkan sebelum data di-pass ke model. Ini dikarenakan model masih belum mengetahui input shape dari data, maka dari itu model menangguhkan inisialisasi parameter, konsep ini juga membuat kita tidak perlu mendefinisikan input shape dari setiap layer, yang mana pastinya sangat membantu terutama jika dataset berupa gambar (yang mana mempunyai shape yang dinamis)

Contohnya seperti kasus berikut, pertama-tama kita membuat model lalu memanggil weight nya

In [16]:
net = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

Kita bisa melihat weightnya masih kosong karena belum diinisialisasikan sama sekali

In [17]:
# Feed model dengan X
net(X)

[w.shape for w in net.get_weights()], net.layers[0].get_weights()[0]

([(4, 256), (256,), (256, 10), (10,)],
 array([[-1.43906385e-01, -1.75253153e-02,  1.23785675e-01, ...,
         -1.39409989e-01, -1.11811608e-01,  6.18231148e-02],
        [ 1.08350545e-01,  1.20449543e-01,  2.19423920e-02, ...,
         -3.19613814e-02,  8.46543759e-02,  4.96828109e-02],
        [ 9.15868580e-03,  1.24413133e-01,  3.32612544e-02, ...,
          1.23169452e-01,  1.28818035e-01,  7.52954483e-02],
        [-1.19774491e-02, -2.49719173e-02,  1.17807865e-01, ...,
         -6.49316981e-02,  1.25780702e-04,  1.46672279e-01]], dtype=float32))

Barulah setelah data di-pass weightnya terinisialisasi. Ini dinamakan Deferred Initialization (penangguhan inisialisasi).

# 4. Custom Layers
Pada kasus-kasus tertentu, kita berhadapan dengan masalah di mana kita membutuhkan layer yang belum pernah dibuat atau belum ada pada library, untuk mengatasi masalah tersebut, kita bisa membuat layer kustom kita sendiri. Membuat layer kustom sama saja seperti membuat blok. Terdapat 2 jenis kustom layer: layer tanpa atau dengan parameter (seperti weight atau bias)

### Layer tanpa parameter


Layer tanpa parameter merupakan layer yang tidak mempunyai weight dan bias di dalamnya (seperti layer flatten). Misal kita ingin membuat layer yang mengurangi input dengan satu

In [19]:
class MinusOneLayer(keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, input):
        return input - tf.ones(input.shape)

In [20]:
layer_wo_params= MinusOneLayer()
layer_wo_params(tf.constant([1.,2.,3.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 2.], dtype=float32)>

Mengimplementasikannya pada model

In [21]:
net = keras.Sequential([keras.layers.Dense(128), MinusOneLayer()])

### Layer dengan parameter
Layer dengan parameter mempunyai dan melibatkan weight dan bias pada operasinya. Misal kita ingin membuat layer dense yang berfungsi sama dengan layer dense pada umumnya (input * weight + bias lalu aktivasi). Walaupun kita mengatur/inisialisasi parameter semau kita, parameter juga tetap berubah saat proses back propagation

In [22]:
class MyDense(keras.Model):
    # Di sini kita berikan dua parameter saja yaitu untuk output dan aktivasi (tentu saja kita bisa menambahkannya sesuai kebutuhan)
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation= activation

    # Pada method build kita inisialisasi weight dan bias, fungsi build hanya dijalankan satu kali
    # Jadi weight dan bias tetap akan ter-update saat back propagation
    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    # Pada fungsi call kita melakukan operasi (input * weight + bias lalu aktivasi)
    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias

        if self.activation is None:
            return linear
        
        return self.activation(linear)

In [23]:
temp_dense= MyDense(5)
temp_dense(X)

temp_dense.get_weights()

[array([[-0.00590205,  0.08091338,  0.02407169,  0.0842525 , -0.01284575],
        [-0.01805593, -0.02711487,  0.03130331, -0.00570907, -0.10003891],
        [-0.03238059, -0.00634598,  0.00644104, -0.02775725,  0.01145109],
        [ 0.06018232,  0.09229618, -0.12428971,  0.03122694, -0.04565911]],
       dtype=float32), array([0., 0., 0., 0., 0.], dtype=float32)]

Selanjutnya membuat model dengan 2 layer MyDense yang telah dibuat

In [24]:
net= keras.Sequential([
    keras.layers.Flatten(),
    MyDense(128, tf.nn.relu), 
    MyDense(64, tf.nn.relu), 
    MyDense(32, tf.nn.relu), 
    MyDense(10, tf.nn.softmax)
])

net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Kita akan mengetes model tersebut pada dataset mnist

In [25]:
(x_train, y_train), (x_test, y_test)= keras.datasets.mnist.load_data()
x_train, x_test= x_train/255, x_test/255

11501568/11490434 [==============================] - 0s 0us/step


In [26]:
net.fit(x_train, y_train, epochs=3, batch_size= 128)

Epoch 1/3
469/469 [==============================] - 2s 3ms/step - loss: 0.5440 - accuracy: 0.8397
Epoch 2/3
469/469 [==============================] - 1s 3ms/step - loss: 0.2055 - accuracy: 0.9405
Epoch 3/3
469/469 [==============================] - 1s 3ms/step - loss: 0.1409 - accuracy: 0.9578


In [27]:
net.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1487 - accuracy: 0.9549


[0.14873085916042328, 0.9549000263214111]

# 5. File I/O

### Save / Load Tensor
Untuk tensor, kita bisa menggunakan numpy.save(namaFile.npy, tensor) untuk menyimpan tensor dan numpy.load(namaFile.npy) untuk memuatnya.

In [28]:
save_x= tf.constant(['a', 'b', 'c'])
save_x

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'a', b'b', b'c'], dtype=object)>

Menyimpan tensor

In [29]:
import numpy as np

np.save('save-x.npy', save_x)

Meload tensor

In [30]:
temp_x= np.load('save-x.npy', allow_pickle=True)
temp_x

array([b'a', b'b', b'c'], dtype=object)

### Save / Load Model Parameter
Misal model kita seperti ini

In [31]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net= MLP()

Lalu kita melakukan feed forward pada model

In [32]:
Y1= net(X)
Y1

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.13764443, -0.10538355,  0.12070184,  0.02399941, -0.07400499,
        -0.1601918 , -0.03911384, -0.01174514,  0.09229151, -0.14785086],
       [-0.05391055, -0.05772851,  0.05543849,  0.03385137, -0.06745581,
        -0.07958122, -0.00766912,  0.027484  ,  0.05311727, -0.04992761]],
      dtype=float32)>

Sekarang kita akan menyimpan parameter model dengan fungsi save_weights('nama_weight.params')

In [33]:
net.save_weights('mlp.params')

Sekarang kita akan membuat model dengan arsitektur yang sama tapi tidak dilakukan feed forward, sebaliknya parameternya diambil/diload dari file parameter yang telah di-save sebelumnya

In [34]:
net_2= MLP()
net_2.load_weights('mlp.params')

# Feed forward
Y2= net_2(X)

Sekarang kita akan membandingkan variabel Y1 dengan Y2

In [35]:
Y1==Y2

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

Hasilnya adalah sama

# 6. GPUs
Mengecek ketersediaan GPU

In [36]:
!nvidia-smi

Sat Apr 23 13:42:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    28W /  70W |    944MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Kita dapat menentukan device, seperti CPU dan GPU, untuk penyimpanan dan komputasi. Secara default, tensor dibuat di memori utama dan menggunakan CPU untuk komputasi.

In [37]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f03ca15bb40>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f03ca15b910>)

Kita bisa menjalankan query untuk mendapatkan berapa banyak GPU dalam sistem

In [38]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

### Tensor dan GPU

Secara default, tensor dibuat dalam CPU, untuk melihat di device mana suatu tensor berada kita bisa menjalankan fungsi seperti ini

In [39]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Outputnya mengatakan GPU karena saya menggunakan google colabs dan telah mengatur runtime pada saat penulisan kode, namun secara default, kalau tidak didefinisikan, akan mengeluarkan output CPU

### Menyetel penggunaan GPU

Misal kita ingin menggunakan GPU pertama atau GPU:0

In [40]:
tf.device('GPU:0')

In [41]:
with tf.device('GPU:0'):
    test = tf.ones((2, 3))

test.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Hati-hati saat pemilihan GPU terutama jika kita memiliki lebih dari satu GPU dalam satu komputer, karena akan terjadi error jika kita melakukan operasi yang melibatkan 2 tensor yang berada di GPU yang berbeda